# One Location, One Temporal Scale, One Operation, Linear Programming Example

## [Example 1]

There is a single process to be optimized for cost [USD] over 4 quarters of a year. 
There is variability in terms of how much of the (known) process [Wind Farm] capacity can be accessed, and resource (Power) demand.


## Initialize

In [7]:
from energia import *
m = Model('small_1L_1T_1O_LP')

## Time

We have 4 quarter which form a year

In [ ]:
m.q = Period()
m.y = 4 * m.q

The horizon ($\overset{\ast}{t} = argmin_{n \in N} |\mathcal{T}_{n}|$) is determined by Energia implicitly. Check it by printing Model.Horizon

In [9]:
m.horizon

y

## Space

If nothing is provided, a default location is created. The created single location will serve as the de facto network

In single location case studies, it may be easier to skip providing a location all together 

In [10]:
m.network 

ntw

## Resources

In the Resource Task Network (RTN) methodology all commodities are resources. In this case, we have a few general resources (wind, power) and a monetary resource (USD).

In [11]:
m.usd = Currency()
m.power, m.wind = Resource(), Resource()

### Setting Bounds

The first bound is set for over the network and year, given that no spatiotemporal disposition is provided: 

$\mathbf{cons}_{wind, network, year_0} \leq 400$ 

For the second bound, given that a nominal is given, the list is treated as multiplicative factor. The length matches with the quarterly scale. Thus:

$\mathbf{rlse}_{power, network, quarter_0} \geq 60$

$\mathbf{rlse}_{power, network, quarter_1} \geq 70$

$\mathbf{rlse}_{power, network, quarter_2} \geq 100$

$\mathbf{rlse}_{power, network, quarter_3} \geq 30$


In [12]:
m.wind.consume <= 400
m.power.release.prep(100) >= [0.6, 0.7, 1, 0.3]

--- General Resource Balance for wind in (ntw, y): initializing constraint , adding consume
--- General Resource Balance for power in (ntw, q): initializing constraint , adding release


Check the model anytime, using m.show(), or object.show()

The first constraint is a general resource balance, generated for every resource at every spatiotemporal disposition at which an aspect regarding it is defined. 

Skip the True in .show() for a more concise set notation based print

In [13]:
m.power.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Process 

There are multiple ways to model this.

Whats most important, however, is the resource balance. The resource in the brackets is the basis resource. For a negative basis, multiply the resource by the negative factor or just use negation if that applies. 

In [14]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind

Now set bounds on the extent to which the wind farm can operate. We are actually writing the following constraint: 

$\mathbf{opr} <= \phi \cdot \mathbf{cap}$

However, we know the capacity, so it is treated as a parameter. 

Note that the incoming values are normalized by default. Use norm = False to avoid that

In [15]:
m.wf.operate.prep(200, norm = False) <= [0.9, 0.8, 0.5, 0.7]
m.wf.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Alternatively, 

In [16]:
# m.wf.capacity == 200 
# m.wf.operate <= [0.9, 0.8, 0.5, 0.7]

Add a cost to the process operation. This implies that the cost of operating is variable in every quarter. In general:

$\dot{\mathbf{v}} == \theta \cdot \mathbf{v}$

In [17]:
m.wf.operate[m.usd.spend] == [4000, 4200, 4300, 3900] 
m.spend.show(True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that printing can be achieved via the aspect [operate, spend, etc.] or the object

## Locating the process

The production streams are only generated once the process is places in some location. In this study, the only location is available is the default network.

In [18]:
m.network.operations(m.wf)

--- General Resource Balance for power in (ntw, q): adding produce
--- General Resource Balance for wind in (ntw, y): adding expend
--- Mapping operate: from (wf, ntw, q) to (wf, ntw, y)


Alternatively,

In [19]:
# m.wf.locate(m.network)

# The Model

The model consists of the following:

1. A general resource balances for wind in (network, year) and power in (network, quarter)
2. Bounds on wind consumption [upper] and power release [lower], and wf operation [upper]
3. Conversion constraints, giving produced and expended resources based on operation
4. Calculation of spending USD in every quarter
5. Mapping constraints for operate: q -> y and spend: q -> y (generated after objective is set)


In [20]:
m.show()

# Mathematical Program for small_1L_1T_1O_LP

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize

For maximization use opt(False)

In [21]:
m.usd.spend.opt()

--- Mapping spend: from (usd, operate, wf, ntw, q) to (usd, ntw, y)
Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file small_1L_1T_1O_LP.mps
Reading time = 0.01 seconds
SMALL_1L_1T_1O_LP: 25 rows, 20 columns, 47 nonzeros
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 25 rows, 20 columns and 47 nonzeros
Model fingerprint: 0xffcbdee5
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 4e+02]
Presolve removed 25 rows and 20 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0810000e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 it

## The Solution

### The overall

In [22]:
m.sol()

# Solution for small_1L_1T_1O_LP

## Objective

<IPython.core.display.Math object>

## Variables

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Individual

The individual solutions can be obtained as list as well (using aslist = True)

In [23]:
m.release.sol(True)

[60.0, 70.0, 100.0, 30.0]